In [1]:
from IPython.display import display, HTML
import scipy.constants as sc
from scipy.optimize import fsolve
from ipywidgets import interact, widgets
%matplotlib notebook

import numpy as np
import cmath
import matplotlib.pyplot as plt
import scipy.constants as sc
plt.rcParams.update({
    "text.usetex": True,
    'text.latex.preamble': r"\usepackage{amsmath} \boldmath"
})
import os
import h5py

print(os.getpid())
%cd ../

8720
/media/work/docs/codes/QuCF/scripts-py


In [2]:
# --- Launch to change cells' width ---
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import subprocess
import pylib.mix as mix
import pylib.qucf_read as qucf_r
import pylib.qucf_oracle as qucf_o
import pylib.qucf_matrix_D as qucf_m

In [4]:
# --------------------------------------------------------------
# --- A helper class to create a diagonal matrix ---
# --------------------------------------------------------------
mix.reload_module(mix)
mix.reload_module(qucf_o)

class DiagMatrix__:
    nz_ = None
    ny_ = None
    nx_ = None
    
    Nz_ = None
    Ny_ = None
    Nx_ = None
    N_ = None
    
    A_ = None
    
    circ_ = None
    
    
    def __init__(self, nz, ny, nx):
        self.nz_ = nz
        self.ny_ = ny
        self.nx_ = nx      
        self.Nz_ = 1<<self.nz_
        self.Ny_ = 1<<self.ny_
        self.Nx_ = 1<<self.nx_  
        self.N_ = self.Nz_ * self.Ny_ * self.Nx_
        
        # initialize the matrix:
        self.A_ = np.zeros((self.N_, self.N_))
        
        # create the circuit:
        self.circ_ = qucf_o.Circuit__()
        anc = qucf_o.Regs__()
        inp = qucf_o.Regs__()
        
        anc.add_reg("azr", 1, True)
        anc.add_reg("ayr", 1, True)
        anc.add_reg("axr", 1, True)
        
        inp.add_reg("rz", self.nz_, False)
        inp.add_reg("ry", self.ny_, False)
        inp.add_reg("rx", self.nx_, False)
        
        self.circ_.set_regs(inp, anc)
        self.circ_.compute_N_registers()
        
        # set the values of the matrix:
        self.create()
        return
    
    
    def get_A(self):
        return self.A_
    
    
    def get_data(self):
        sizes = [self.Nz_, self.Ny_, self.Nx_]
        return [self.circ_, self.A_, sizes]
    
    
    def set_v(self, v, iz, iy, ix):
        ir = iz * self.Ny_*self.Nx_ + iy*self.Nx_ + ix
        self.A_[ir,ir] = v
        return
    
    
    def create(self):
        Nz, Ny, Nx = self.Nz_, self.Ny_, self.Nx_
        
        # Set-z-1
        for iz in range(Nz//2):
            for iy in range(Ny//4):
                ix, v = 0, 0.3
                self.set_v(v, iz, iy, ix)
                for ix in range(1,Nx):
                    v = 0.4
                    self.set_v(v, iz, iy, ix)

            for iy in range(Ny//4,Ny//2):
                ix, v = 0, 0.42
                self.set_v(v, iz, iy, ix)
                for ix in range(1,Nx-1):
                    v = 0.46
                    self.set_v(v, iz, iy, ix)
                ix, v = Nx-1, 0.21
                self.set_v(v, iz, iy, ix)

            for iy in range(Ny//2, 3*Ny//4):
                ix, v = 0, 0.6
                self.set_v(v, iz, iy, ix)
                ix, v = 1, 0.48
                self.set_v(v, iz, iy, ix)
                for ix in range(2,Nx):
                    v = 0.422
                    self.set_v(v, iz, iy, ix)

            for iy in range(3*Ny//4,Ny):
                for ix in range(Nx):
                    v = 0.1
                    self.set_v(v, iz, iy, ix)

        # Set-z-2    
        for iz in range(Nz//2, Nz):
            xx = 0

            for iy in range(3*Ny//4):
                for ix in range(Nx//2):
                    v = 0.1
                    self.set_v(v, iz, iy, ix)
                for ix in range(Nx//2,Nx):
                    v = 0.2
                    self.set_v(v, iz, iy, ix)

            for iy in range(3*Ny//4,Ny):
                for ix in range(Nx-1):
                    v = 0.8
                    self.set_v(v, iz, iy, ix)
                ix, v = Nx-1, 0.02
                self.set_v(v, iz, iy, ix)
        return
        

In [21]:
# --------------------------------------------------------------
# --- Chose the case ---
# --------------------------------------------------------------
mix.reload_module(mix)

D1 = DiagMatrix__(3, 3, 2)
D2 = DiagMatrix__(4, 3, 2)
D3 = DiagMatrix__(3, 4, 2)
D4 = DiagMatrix__(3, 3, 3)

nz_large, ny_large, nx_large = 5, 5, 4
D_ref = DiagMatrix__(nz_large, ny_large, nx_large)

In [22]:
# ---------------------------------------------------------------
# --- Initialize matrices D_F for various sizes ---
# ---------------------------------------------------------------
mix.reload_module(qucf_m)

grid_332 = qucf_m.SectionsGrid__(D1.get_data())
grid_432 = qucf_m.SectionsGrid__(D2.get_data())
grid_342 = qucf_m.SectionsGrid__(D3.get_data())
grid_333 = qucf_m.SectionsGrid__(D4.get_data())

In [23]:
# ---------------------------------------------------------------
# --- Create an extrapolation template ---
# ---------------------------------------------------------------
mix.reload_module(qucf_m)

oo_extr = qucf_m.Extrapolation__([grid_332, grid_432, grid_342, grid_333])
oo_extr.create_extrapolation_template()

All grids have the same number of sizes.
The correct number of grids is provided.
Grids have similiar structure.


In [25]:
# ---------------------------------------------------------------
# --- Construct a large matrix ---
# ---------------------------------------------------------------
mix.reload_module(qucf_m)
mix.reload_module(qucf_o)

def create_circ_large(nz, ny, nx):
    circ = qucf_o.Circuit__()
    anc = qucf_o.Regs__()
    inp = qucf_o.Regs__()

    anc.add_reg("azr", 1, True)
    anc.add_reg("ayr", 1, True)
    anc.add_reg("axr", 1, True)

    inp.add_reg("rz", nz, False)
    inp.add_reg("ry", ny, False)
    inp.add_reg("rx", nx, False)

    circ.set_regs(inp, anc)
    circ.compute_N_registers()
    return circ
# -----------------------------------------
circ_large = create_circ_large(nz_large, ny_large, nx_large)
A_large = oo_extr.reconstruct_matrix(circ_large)

In [28]:
# ----------------------------------------------------------------------
# --- Compare the reconstructed and calculated (if available) matrix ---
# ----------------------------------------------------------------------
mix.reload_module(qucf_o)
qucf_o.compare_matrices(A_large, D_ref.get_A())

the same within precision  1e-06
